In [14]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load the tokenizer from the saved files
tokenizer = GPT2Tokenizer.from_pretrained('model_tokenizer')

# Instantiate the model with the saved configuration
config = GPT2Config.from_pretrained('model_tokenizer')
model = GPT2ForSequenceClassification(config)

# Load the trained model weights from the .pth file
model.load_state_dict(torch.load('gpt2_sentiment_analysis.pth', map_location=torch.device('cpu')))

# Set the model to evaluation mode
model.eval()  
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Define a prediction function
def predict_headline(headline, tokenizer, model, device):
    model.eval()  # Ensure the model is in evaluation mode
    inputs = tokenizer.encode_plus(
        headline, 
        return_tensors="pt", 
        max_length=512, 
        truncation=True, 
        padding="max_length"
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=-1)
        
    return "Increase" if prediction == 1 else "Decrease"

# Example usage
# headline = "everythin is bad and the world is ending"
# print(predict_headline(headline, tokenizer, model, torch.device("cpu")))

# Testing the model with a small sample from the validation set
test_sample = val_df.sample(n=5).reset_index(drop=True)
test_data_loader = create_data_loader(test_sample, tokenizer, MAX_LEN, 1)  # Single batch size for individual assessment

predictions, real_values = evaluate(model, test_data_loader, device)

# Print predictions and actual labels
for i in range(len(predictions)):
    print(f"Headline: {test_sample['All_Headlines'].iloc[i]}")
    print(f"Predicted sentiment: {'Increase' if predictions[i] == 1 else 'Decrease'}")
    print(f"Actual movement: {'Increase' if real_values[i] == 1 else 'Decrease'}")
    print("---")


NameError: name 'val_df' is not defined